In [6]:
url_betting_wc = "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=12357;type=tournament"
soup_betting_wc = BeautifulSoup(requests.get(url_betting_wc).text, "lxml")
s_batting_wc = soup_betting_wc.select("table", {"class" : "engineTable"})[0].find_all("tbody")[0].find_all("tr", {"class" : "data1"})
players_links = []
countries = []
players_names = []
mapping = {"AFG" : "afghanistan",  "ENG" : "england",  "WI"  : "westindies",  "SA"  : "southafrica",
           "PAK" : "pakistan",  "AUS" : "australia",  "NZ"  : "newzealand",  "INDIA" : "india",
           "SL"  : "srilanka",  "BDESH" : "bangladesh"}
for i in s_batting_wc:
    WC_batting_data = i.get_text().splitlines()[1:]
    country = i.get_text().splitlines()[1]
    m = re.search(r"\((\w+)\)", country)
#     player_link = "http://www.espncricinfo.com/" + mapping[m.group(1)] + i.find("a", {"class" : "data-link"})['href'][3:]
#     players_links.append(player_link)
    countries.append(mapping[m.group(1)])
#     soup_player = BeautifulSoup(requests.get(player_link).text, "lxml")
#     player_name = soup_player.find("div", {"class" : "ciPlayernametxt"}).find("h1").find("a")["alt"]
#     player_name = player_name.replace("RSS Feed", "").strip()
#     players_names.append(player_name)
for i in ['countries']:#, 'players_links', 'players_names']:
    import pickle
    with open(f"{i}.pkl", "wb") as file:
        pickle.dump(eval(i), file)

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import pickle
os.chdir("all-privous-data")
with open("countries.pkl", "rb") as file:
    countries = pickle.load(file)
with open("players_links.pkl", "rb") as file:
    players_links = pickle.load(file)
with open("players_names.pkl", "rb") as file:
    players_names = pickle.load(file)
        


for url, plyr_name, country in zip(players_links, players_names, countries):
    if not country in os.listdir():
        os.makedirs(country)
    a = url[url.rfind("/")+1:url.rfind(".html")]
    modified_url = f"http://stats.espncricinfo.com/ci/engine/player/{a}.html?class=2;template=results;type=allround"
    soup = BeautifulSoup(requests.get(modified_url).text, "lxml") 
    b = soup.find("div", {"class" : "pnl650M"})
    c = b.select("table", {"class" : "engineTable"})[3]
    columns = [i for i in c.find("thead").text.splitlines() if i]
    rows = []
    for row in c.select("tbody")[1:]:
        for z in row.select("tr", {"class" : "data1"}):
            d = z.text.splitlines()
            if d:
                rows.append(d[1:-1])
    ddf = pd.DataFrame(rows, columns=columns).dropna(how='all')
    ddf.to_csv(country + "/" + plyr_name + ".csv")

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [8]:
# for i in ['players_links', 'countries', 'players_names']:
#     import pickle
#     with open(f"{i}.pkl", "wb") as file:
#         pickle.dump(eval(i), file)

# WC datrailed bowling data

In [89]:
for url, plyr_name, country in zip(players_links, players_names, countries):
    if not country+"_WC" in os.listdir():
        os.makedirs(country+"_WC")
    player_number = url[url.rfind("/")+1:url.rfind(".html")]
    WC_url = f"http://stats.espncricinfo.com/ci/engine/player/{player_number}.html?class=2;template=results;trophy=12;type=allround;view=innings"
    soup = BeautifulSoup(requests.get(WC_url).text, "lxml")
    columns = [i for i in soup.find("div", {"class" : "pnl650M"}).select("table", {"class" : "engineTable"})[3].find("thead").text.splitlines() if i]
    columns = columns[:7] + ["empty"] + columns[7:]
    s = soup.find("div", {"class" : "pnl650M"}).select("table", {"class" : "engineTable"})[3].find("tbody").find_all("tr")
    rows = []
    for i in s:
        rows.append(i.get_text().splitlines()[1:-1])
    adf = pd.DataFrame(rows, columns=columns).head()
    adf.to_csv(country +"_WC/" + plyr_name)